In [41]:
# Installing necessary packages
!pip install keras-tuner

# Importing necessary libraries
import os
import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from xgboost import XGBClassifier

import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.models import load_model, save_model

import tensorflow as tf


In [ ]:
df = pd.read_csv('liberman_manor_marcotti.csv')

grouped = df.groupby(['filename', 'Freq(Hz)'])

/var/folders/bp/pzmg_4bx0kxc4_jr5p1npj1h0000gn/T/ipykernel_20875/3423745150.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../liberman_manor_marcotti.csv')


In [43]:
# Create a unique identifier for each file-frequency combination
df['file_freq'] = df['filename'].astype(str) + df['Freq(Hz)'].astype(str)
unique_file_freq = df['file_freq'].unique()

# Function to normalize each group by 'file_freq'
def normalize_group(group):
    # Drop columns that shouldn't be scaled
    # Delete 'Hearing' column
    try:
        data_to_scale = group.drop(columns=['Hearing', 'filename', 'file_freq', 'Freq(Hz)', 'Level(dB)'])
    except KeyError:
        data_to_scale = group.drop(columns=['filename', 'file_freq', 'Freq(Hz)', 'Level(dB)'])

    # Flatten the data to scale all values across the group
    flattened_data = data_to_scale.values.flatten().reshape(-1, 1)
    
    # Step 1: Standardize the data
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(flattened_data)

    # Step 2: Apply min-max scaling
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))  # Adjust range if needed
    scaled_data = min_max_scaler.fit_transform(standardized_data).reshape(data_to_scale.shape)

    # Reshape back to the original shape and update the group
    group[data_to_scale.columns] = scaled_data

    return group

# Normalize test set
df = df.groupby('file_freq').apply(normalize_group)

try:
    df_inference = df.drop(columns=['Hearing', 'filename', 'file_freq', 'Freq(Hz)', 'Level(dB)'])
except KeyError:
    df_inference = df.drop(columns=['filename', 'file_freq', 'Freq(Hz)', 'Level(dB)'])

y = df['Hearing']

# Reshape data for CNN
inference_cnn = np.expand_dims(df_inference.values, axis=2)

In [44]:
df

filename  Hearing  Freq(Hz)  Level(dB)         0         1  \
file_freq                                                                       
100100.0     17244      100        0     100.0        0.0  0.433297  0.433951   
             17245      100        0     100.0        5.0  0.453915  0.451586   
             17246      100        0     100.0       10.0  0.456277  0.457584   
             17247      100        0     100.0       15.0  0.444246  0.454224   
             17248      100        0     100.0       20.0  0.452140  0.452570   
...                     ...      ...       ...        ...       ...       ...   
WPZ1738000.0 6833    WPZ173        1    8000.0       45.0  0.296125  0.320917   
             6834    WPZ173        1    8000.0       50.0  0.259951  0.260173   
             6835    WPZ173        1    8000.0       60.0  0.277257  0.227402   
             6836    WPZ173        1    8000.0       70.0  0.327851  0.332164   
             6837    WPZ173        1    8000.0       80.0  0.447159  0.433485   

                           2         3         4         5  ...       235  \
file_freq                                                   ...             
100100.0     17244  0.434686  0.433941  0.433697  0.437359  ...  0.459934   
             17245  0.452399  0.455676  0.458698  0.456973  ...  0.443450   
             17246  0.459806  0.460652  0.461606  0.461010  ...  0.422313   
             17247  0.461965  0.462652  0.457173  0.449367  ...  0.425833   
             17248  0.452215  0.451581  0.449192  0.446553  ...  0.426648   
...                      ...       ...       ...       ...  ...       ...   
WPZ1738000.0 6833   0.337885  0.336426  0.329509  0.336157  ...  0.320592   
             6834   0.285449  0.295270  0.309839  0.285695  ...  0.339514   
             6835   0.236448  0.254242  0.251402  0.252191  ...  0.292451   
             6836   0.345794  0.332713  0.296292  0.261242  ...  0.309987   
             6837   0.380503  0.293021  0.239307  0.222654  ...  0.305405   

                         236       237       238       239       240  \
file_freq                                                              
100100.0     17244  0.454084  0.450123  0.444532  0.440080  0.440417   
             17245  0.444162  0.443467  0.440450  0.440771  0.443532   
             17246  0.419870  0.421496  0.423810  0.423040  0.425274   
             17247  0.430533  0.435900  0.442050  0.445494  0.447072   
             17248  0.424831  0.422138  0.421158  0.422080  0.422639   
...                      ...       ...       ...       ...       ...   
WPZ1738000.0 6833   0.320846  0.326508  0.331149  0.335299  0.325144   
             6834   0.337453  0.341744  0.354379  0.358014  0.359686   
             6835   0.287675  0.303424  0.322957  0.330439  0.329280   
             6836   0.315659  0.317036  0.354243  0.319768  0.280805   
             6837   0.299788  0.303645  0.428848  0.394715  0.283119   

                         241       242       243     file_freq  
file_freq                                                       
100100.0     17244  0.442441  0.445606  0.445309      100100.0  
             17245  0.447215  0.444228  0.442798      100100.0  
             17246  0.428013  0.426424  0.425272      100100.0  
             17247  0.449588  0.450137  0.449236      100100.0  
             17248  0.421666  0.422442  0.422801      100100.0  
...                      ...       ...       ...           ...  
WPZ1738000.0 6833   0.326925  0.323461  0.301887  WPZ1738000.0  
             6834   0.371022  0.390091  0.395268  WPZ1738000.0  
             6835   0.339201  0.324470  0.300130  WPZ1738000.0  
             6836   0.324020  0.333611  0.278501  WPZ1738000.0  
             6837   0.378882  0.366718  0.252549  WPZ1738000.0  

[23328 rows x 249 columns]

In [45]:
model = load_model('../models/abr_thresholding.keras')

In [46]:
# Predictions
predictions = (model.predict(inference_cnn) > 0.5).astype(int)

test_data_with_predictions = df.assign(prediction=predictions)[['filename', 'prediction', 'Freq(Hz)', 'Level(dB)']]

729/729 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step


In [47]:
_, accuracy = model.evaluate(inference_cnn, y)
print('Accuracy: %.2f' % (accuracy*100))

729/729 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9666 - loss: 0.0829
Accuracy: 96.06


In [48]:
test_data_with_predictions

filename  prediction  Freq(Hz)  Level(dB)
file_freq                                                   
100100.0     17244      100           0     100.0        0.0
             17245      100           0     100.0        5.0
             17246      100           0     100.0       10.0
             17247      100           0     100.0       15.0
             17248      100           0     100.0       20.0
...                     ...         ...       ...        ...
WPZ1738000.0 6833    WPZ173           1    8000.0       45.0
             6834    WPZ173           1    8000.0       50.0
             6835    WPZ173           1    8000.0       60.0
             6836    WPZ173           1    8000.0       70.0
             6837    WPZ173           1    8000.0       80.0

[23328 rows x 4 columns]

In [ ]:
thresholds = []

for (filename, freq), group in test_data_with_predictions.groupby(['filename', 'Freq(Hz)']):
    group = group.sort_values('Level(dB)', ascending=False)
    
    threshold_db = None
    previous_prediction = None

    for p, d in zip(group['prediction'], group['Level(dB)']):
        if p == 0:
            if previous_prediction == 0:
                break  # Stop if two consecutive 0s
            previous_prediction = 0
        else:
            threshold_db = d  # Update threshold on any 1
            previous_prediction = 1

    if threshold_db is None:
        threshold_db = group['Level(dB)'].max()  # All 0s fallback

    thresholds.append({
        'filename': filename,
        'Freq(Hz)': freq,
        'predicted_threshold': threshold_db
    })

In [ ]:
thresholds = pd.DataFrame(thresholds)

In [ ]:
thresholds

,filename,Freq(Hz),predicted_threshold
0,1,100.0,30.0
1,1,3000.0,75.0
2,1,6000.0,40.0
3,1,12000.0,25.0
4,1,18000.0,30.0
...,...,...,...
1518,WPZ173,11300.0,20.0
1519,WPZ173,16000.0,25.0
1520,WPZ173,22600.0,25.0
1521,WPZ173,32000.0,20.0
